# Calculate robusteness from pi control runs

In [21]:
# Some definitions

# directory of data
#fdir = '~/Google Drive/professional/research/FARALLON_INSTITUTE_PROJECTS/2020 NOAA MAPP/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'
fdir = '/Volumes/GoogleDrive/My Drive/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'

In [22]:
# Modules
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore') #filter some warning messages
import seaborn as sns

In [23]:
def num_coevents(y1, y2, pct1, pct2, runs='picontrol'):
    # 
    fin1 = fdir+'anomaly ts/'+runs+'_sst_anom_500yr.nc'
    fin2 = fdir+'anomaly ts/'+runs+'_sm_anom_500yr.nc'
    modsst = xr.open_dataset(fin1)
    modsst.close()
    modsm2 = xr.open_dataset(fin2)
    modsm2.close()
    
    models = modsm2.model

    # selec periood
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    
    nev = list()
    
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
    
        # calculate threshold
        mhw_thr = np.percentile(tmp1,pct1)
        drg_thr = np.percentile(tmp2,pct2)
    
        a1 = tmp1>=mhw_thr
        a2 = tmp2<=drg_thr
        
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]
    
        nev.append(len(tmp)/30) #freq.
        
    return nev  # sum(nev)

In [24]:
pinev=list()
for i in range(int(502/30)):
    nev = num_coevents(i*30, (i+1)*30-1, 90, 10)
    pinev.append(nev)
pinev 

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.03333333333333333,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.03333333333333333,
  0.0,
  0.0,
  0.03333333333333333,
  0.0,
  0.0,
  0.03333333333333333,
  0.0,
  0.0,
  0.03333333333333333,
  0.03333333333333333],
 [0.0,
  0.0,
  0.03333333333333333,
  0.03333333333333333,
  0.0,
  0.0,
  0.03333333333333333,
  0.03333333333333333,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.03333333333333333,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.03333333333333333,
  0.0],
 [0.0,
  0.0,
  0.033333333333333

In [25]:
#model by model

In [26]:
model_mean=np.mean(pinev,axis=0)
model_spread=np.std(pinev,axis=0)
model_mean

array([0.00208333, 0.00416667, 0.00416667, 0.00416667, 0.00833333,
       0.00208333, 0.00833333, 0.0125    , 0.00416667, 0.        ,
       0.00625   , 0.00625   , 0.00416667, 0.00625   , 0.00625   ,
       0.00416667, 0.        , 0.00416667, 0.00625   , 0.00625   ,
       0.00416667, 0.00833333])

In [27]:
#16 30-yr periods MMEs

In [28]:
mme_mean=np.mean(pinev,axis=1)
mme_spread=np.std(pinev,axis=1)
mme_mean

array([0.        , 0.        , 0.        , 0.00909091, 0.00909091,
       0.01363636, 0.00606061, 0.00909091, 0.        , 0.        ,
       0.        , 0.        , 0.01060606, 0.00909091, 0.00454545,
       0.01060606])

In [29]:
#All model and periods

In [30]:
cross_model_mean=np.mean(model_mean)
cross_model_spread=np.std(model_mean)
cross_model_mean,cross_model_spread

(0.0051136363636363645, 0.002789946943684893)

In [31]:
## Calculate thresholds for PI Control runs

In [32]:
def get_thre(fdir, runs, y1,y2,prc1,prc2):
    # open data: anomalies (without trend)

    modsst = xr.open_dataset(fdir+'anomaly ts/'+runs+'_sst_anom.nc')
    modsst.close()
    modsm2 = xr.open_dataset(fdir+'anomaly ts/'+runs+'_sm_anom.nc')
    modsm2.close()

    # select period 
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    mhw_thr = list()
    drg_thr = list()
    models = list()

    # Calculate thresholds for each model
    for ix,i in enumerate(modsm2.model.values):
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
        
        mhw_thr.append(np.nanpercentile(tmp1,prc1))
        drg_thr.append(np.nanpercentile(tmp2,prc2))

        models.append(i)
    
    return mhw_thr, drg_thr, models

In [33]:
# calculate number of events for control, historical and future
# based on thresholds in each run

def freq_coevents(fdir, runs, mhw_thr, drg_thr, models, y1, y2):
    #pi control
    fi1 = fdir+'anomaly ts/'+runs+'_sst_anom.nc'
    fi2 = fdir+'anomaly ts/'+runs+'_sm_anom.nc'
    modsst = xr.open_dataset(fi1)
    modsst.close()
    modsm2 = xr.open_dataset(fi2)
    modsm2.close()

    # select period
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))

    nev = list()

    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values

        a1 = tmp1>=mhw_thr[ix]
        a2 = tmp2<=drg_thr[ix]
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]

        nev.append(len(tmp))
    
    freq = np.array(nev)/(y2-y1+1)
    
    return freq, np.round(np.nanmean(freq),4), np.round(np.nanstd(freq),4) #mean frequency
                 #np.round(np.nanmean(nev),4), np.round(np.nanstd(nev),4) #mean no. of events

In [34]:
def freq_difruns(fdir,ny, prc1, prc2):
    
    # calculate threshold in pi control
    y2=2014
    y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(fdir, 'picontrol',y1,y2,prc1,prc2)

    print ('Percentiles: '+str(prc1)+'/'+str(prc2))
    # pi control
    print('piControl')
    y2=2014
    y1=y2-ny+1
    freq, mfrq, sfrq = freq_coevents(fdir,'picontrol',mhw_thr,drg_thr, models,y1,y2)
    print(mfrq,sfrq)
    
    # calculate threshold in historical 1
    y1 = 1900
    y2 = y1+ny-1
    mhw_thr, drg_thr, models = get_thre(fdir, 'historical',y1,y2,prc1,prc2)

    # historical 1
    y1 = 1900
    y2 = y1+ny-1
    print('\nhistorical: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

       
    # calculate threshold in historical 2
    y2=2014
    y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(fdir, 'historical',y1,y2,prc1,prc2)

    # historical 2
    y2=2014
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq1, mfrq, sfrq = freq_coevents(fdir,'historical',mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)
    
    # calculate threshold in historical 2
    y2=2099
    y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(fdir, 'ssp585',y1,y2,prc1,prc2)

    # future
    y2=2099
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq2, mfrq, sfrq = freq_coevents(fdir,'ssp585',mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)
    return freq1, freq2

In [35]:
# Frequency of events - detrended data, using individual threshols
## percentiles 90-10

In [36]:
ny = 30
prc1 = 90
prc2 = 10

print ('\nPercentiles: '+str(prc1)+'/'+str(prc2))
print('\nProbable # events in '+str(ny)+' years'+': '+str(ny*0.01))

hinev,funev=freq_difruns(fdir,ny, prc1, prc2)
funev


Percentiles: 90/10

Probable # events in 30 years: 0.3
Percentiles: 90/10
piControl
0.0061 0.0129

historical: 1900-1929
0.0045 0.0114

historical 2: 1985-2014
0.0076 0.0172

future: 2070-2099
0.0091 0.0148


array([0.        , 0.        , 0.03333333, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.03333333, 0.        ,
       0.03333333, 0.03333333, 0.        , 0.03333333, 0.        ,
       0.03333333, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])

In [37]:
#model by model diff and spread

In [38]:
diff=list()
zip_obj=zip(funev,model_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff

[-0.0020833333333333333,
 -0.004166666666666667,
 0.029166666666666667,
 -0.004166666666666667,
 -0.008333333333333333,
 -0.0020833333333333333,
 -0.008333333333333333,
 -0.012499999999999999,
 0.029166666666666667,
 0.0,
 0.027083333333333334,
 0.027083333333333334,
 -0.004166666666666667,
 0.027083333333333334,
 -0.00625,
 0.029166666666666667,
 0.0,
 -0.004166666666666667,
 -0.00625,
 -0.00625,
 -0.004166666666666667,
 -0.008333333333333333]

In [39]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.003977272727272727, 0.01504817981665247)